# Feature Importance Methods for Scientific Inference — SOLUTIONS

---

## Setup

Run all cells in this section before starting the exercises.

In [ ]:
!pip install fippy -q

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import warnings
warnings.filterwarnings('ignore')

plt.rcParams.update({
    'font.size':        14,
    'axes.titlesize':   16,
    'axes.labelsize':   14,
    'xtick.labelsize':  13,
    'ytick.labelsize':  13,
})

### Data

We generate a dataset with 5 features and a continuous target variable $Y$.
**The data-generating process is hidden for now.**

In [ ]:
def generate_data(n=1500, seed=83):
    rng = np.random.RandomState(seed)
    x1 = rng.normal(0, 1, n)
    x2 = 0.999 * x1 + np.sqrt(1 - 0.999**2) * rng.normal(0, 1, n)
    x3 = rng.normal(0, 1, n)
    x4 = 0.999 * x3 + np.sqrt(1 - 0.999**2) * rng.normal(0, 1, n)
    y  = 5 * x1 + rng.normal(0, 1, n)
    x5 = rng.normal(0, 1, n)
    X  = np.column_stack([x1, x2, x3, x4, x5])
    return X, y

X, y = generate_data()
feature_names = ["X1", "X2", "X3", "X4", "X5"]
print(f"Dataset: {X.shape}, features: {feature_names}")

### Model

We train an OLS linear regression on 1000 training observations and evaluate on 500 held-out test observations.

In [ ]:
n_train = 1000
X_train, X_test = X[:n_train], X[n_train:]
y_train, y_test = y[:n_train], y[n_train:]

model = LinearRegression().fit(X_train, y_train)
r2  = model.score(X_test, y_test)
print(f"Test R\u00b2: {r2:.3f}")
print(f"Coefficients: {np.round(model.coef_, 2)}")

### fippy setup

We use the [`fippy`](https://github.com/gcskoenig/fippy) package for Exercises 2 and 3.
The **Gaussian sampler** estimates the conditional distribution $P(X_j \mid X_{-j})$ in closed form
using the multivariate normal assumption.

In [ ]:
from fippy.explainers import Explainer
from fippy.samplers import GaussianSampler

# fippy requires pandas DataFrames
X_train_df = pd.DataFrame(X_train, columns=feature_names)
X_test_df  = pd.DataFrame(X_test,  columns=feature_names)
y_train_s  = pd.Series(y_train, name='y')
y_test_s   = pd.Series(y_test,  name='y')

sampler   = GaussianSampler(X_train_df)
explainer = Explainer(model.predict, X_train_df,
                      loss=mean_squared_error, sampler=sampler)

---

# Exercise 1: PFI — Implementation and Interpretation

**True DGP:** $Y = 5X_1 + \varepsilon_Y$, with $X_2 \approx X_1$ and $X_4 \approx X_3$, and $X_5$ independent of everything.

In [ ]:
def my_pfi(model, X, y, feature_idx, n_repeats=50, seed=42):
    """
    Permutation Feature Importance for a single feature.

    PFI_j = mean_r [ L(y, f(X_perm_r)) ] - L(y, f(X))
    """
    rng = np.random.RandomState(seed)
    baseline_mse = mean_squared_error(y, model.predict(X))

    perturbed_mses = []
    for _ in range(n_repeats):
        X_perm = X.copy()
        X_perm[:, feature_idx] = rng.permutation(X[:, feature_idx])
        perturbed_mses.append(mean_squared_error(y, model.predict(X_perm)))

    return np.mean(perturbed_mses) - baseline_mse


# Compute and plot PFI for all features
pfi_scores = [my_pfi(model, X_test, y_test, j) for j in range(len(feature_names))]
for name, score in zip(feature_names, pfi_scores):
    print(f"PFI({name}): {score:.4f}")

plt.figure(figsize=(6, 4))
plt.barh(feature_names[::-1], pfi_scores[::-1], color='grey', edgecolor='black', linewidth=0.5)
plt.xlabel("PFI (increase in MSE)")
plt.title("Permutation Feature Importance")
plt.tight_layout()
plt.show()

In [ ]:
# Scatterplot: X3 vs X4 before and after permuting X3
rng = np.random.RandomState(42)
X_perm = X_test.copy()
X_perm[:, 2] = rng.permutation(X_test[:, 2])

fig, axes = plt.subplots(1, 2, figsize=(10, 4))

axes[0].scatter(X_test[:, 2], X_test[:, 3], alpha=0.3, s=10, color='grey')
axes[0].set(xlabel="$X_3$", ylabel="$X_4$", title="Original: $(X_3, X_4)$",
            xlim=(-4,4), ylim=(-4,4))

axes[1].scatter(X_perm[:, 2], X_perm[:, 3], alpha=0.3, s=10, color='grey')
axes[1].set(xlabel=r"$\tilde{X}_3$ (permuted)", ylabel="$X_4$",
            title="After permuting $X_3$", xlim=(-4,4), ylim=(-4,4))

plt.tight_layout()
plt.show()

### Solution: Why is PFI misleading here?

The fitted coefficients are approximately $\hat{\beta} = [3.11,\; 1.88,\; -2.11,\; 2.17,\; 0.02]$.
Due to the near-perfect correlation $\rho(X_3, X_4) = 0.999$, OLS assigns large **opposing** coefficients
to $X_3$ and $X_4$. In the original data these almost cancel. After permuting $X_3$, the cancellation
breaks, causing large prediction errors — which PFI misinterprets as feature importance.

PFI measures **model reliance**, not association with $Y$. Non-zero PFI does not imply $X_j \not\perp Y$.

---

# Exercise 2: Conditional Feature Importance (CFI)

In [ ]:
# Conditional Feature Importance via fippy
ex_cfi = explainer.cfi(X_test_df, y_test_s, nr_runs=10)
ex_cfi.hbarplot()
plt.show()

means, stds = ex_cfi.fi_means_stds()
print("\nCFI scores:")
for feat, m, s in zip(feature_names, means, stds):
    print(f"  {feat}: {m:.4f} ± {s:.4f}")

### Solution: CFI interpretation

CFI correctly assigns near-zero importance to $X_2$ (redundant given $X_1$), $X_3$, $X_4$
(irrelevant collinear), and $X_5$ (purely irrelevant). Only $X_1$ receives a non-zero score,
reflecting that it is the only feature with a **conditional association** with $Y$ given all others.

| Feature | Role | CFI |
|---|---|---|
| $X_1$ | Direct cause of $Y$ | **Non-zero** |
| $X_2$ | Correlated with $X_1$, no direct effect | $\approx 0$ |
| $X_3, X_4$ | Irrelevant, mutually correlated | $\approx 0$ |
| $X_5$ | Purely irrelevant | $\approx 0$ |

---

# Exercise 3: Leave-One-Covariate-Out (LOCO)

In [ ]:
# Compute v(S) for all features and for each leave-one-out set
N = feature_names  # full coalition

ex_vN = explainer.csagevf(S=list(N), X_eval=X_test_df, y_eval=y_test_s)
means_N, _ = ex_vN.fi_means_stds()
v_N = float(np.array(means_N).flatten()[0])
print(f"v(all features) = {v_N:.4f}  [≈ Var(Y)·R²]\n")

loco_scores = {}
for feat in N:
    N_minus_j = [f for f in N if f != feat]
    ex_vNj = explainer.csagevf(S=N_minus_j, X_eval=X_test_df, y_eval=y_test_s)
    means_Nj, _ = ex_vNj.fi_means_stds()
    v_Nj = float(np.array(means_Nj).flatten()[0])
    loco_scores[feat] = v_N - v_Nj
    print(f"LOCO({feat}): {loco_scores[feat]:.4f}  "
          f"({100 * loco_scores[feat] / v_N:.1f}% of explained variance)")

# Plot as share of explained variance
pct   = [100 * loco_scores[f] / v_N for f in N]
plt.figure(figsize=(6, 4))
plt.barh(N[::-1], pct[::-1], color='grey', edgecolor='black', linewidth=0.5)
plt.xlabel("Share of explained variance (%)")
plt.title("LOCO (conditional marginalization)")
plt.axvline(0, color='black', linewidth=0.8)
plt.tight_layout()
plt.show()

### Solution: LOCO interpretation

LOCO expresses importance as a **fraction of the model's $R^2$**.

- $X_1$ accounts for nearly 100% of explained variance — removing it collapses the model.
- All other features contribute $\approx 0\%$, since the conditional distribution can
  reconstruct their contributions from the remaining features.

LOCO $\neq 0$ is equivalent to $X_j \not\perp\!\!\!\perp Y \mid X_{-j}$ (conditional dependence).
Unlike CFI, it also quantifies **how much** of the explained variance each feature is responsible for.